In [ ]:
import time
import sys
import os
import numpy as np

%load_ext autoreload
%autoreload 2

import xfmkit.entry_processed as entry_processed
import xfmkit.entry_raw as entry_raw
import xfmkit.utils as utils
import xfmkit.argops as argops
import xfmkit.clustering as clustering
import xfmkit.visualisations as vis
import xfmkit.processops as processops
import xfmkit.structures as structures
import xfmkit.config as config

#add the parent folder to sys path so we can import from within the notebook subfolder
#sys.path.insert(0,'..')

print(sys.path)

#args = ' -d /home/lachlan/CODEBASE/xfmkit/data/processed_maps/carlos_full -n 2 -fc'
#args_in = ["-d", "/home/lachlan/CODEBASE/xfmkit/data/processed_maps/carlos_full"]
args_in = [ "-f", "/home/lachlan/CODEBASE/xfmkit/data/example_datafile.GeoPIXE", "-a", "-m", "-dt", "-1" ]

PACKAGE_CONFIG='xfmkit/config.yaml'


In [ ]:
pxs, xfmap, = entry_raw.read_raw(args_in)
#pxs, embedding, categories, classavg, palette, kde = entry_processed.read_processed(args_in)

In [ ]:
pxs.dtmod

In [ ]:
import xfmkit.dtops as dtops

dtops.predhist(pxs.dt, pxs.dtmod, '.', pxs.ndet)

In [ ]:
pxs.dtmod.shape[1]

In [ ]:
sample = kde.kde.sample(100000)

sclassifier, scategories = clustering.classify(sample, eom=False)


vis.seaborn_embedplot(sample, scategories, palette=palette)

In [ ]:
import time
import os
import re
import hdbscan
import numpy as np
import umap.umap_ as umap
import pacmap
import pickle

from sklearn import decomposition
from sklearn.cluster import KMeans
from sklearn.neighbors import KernelDensity

import xfmkit.utils as utils
import xfmkit.config as config


def localclassify(embedding, use_classifier):
    """
    performs classification on embedding to produce final clusters

    args:       set of 2D embedding matrices (shape [nreducers,x,y]), number of pixels in map
    returns:    category-by-pixel matrix, shape [nreducers,chan]
    """
    print("RUNNING CLASSIFIER")
    operator, args = use_classifier

    print(f"operator: {operator}")
    print(f"args: {args}")

    classifier = operator(**args)
    embedding = classifier.fit(embedding)

    categories=classifier.labels_

    categories = categories.astype(np.int32)

    categories=categories+1  

    return classifier, categories


In [ ]:
use_classifier_eom = (hdbscan.HDBSCAN, {"min_cluster_size": 100,
        "min_samples": 500,  #500
        "alpha": 1.0,   #1.0
        "cluster_selection_epsilon": 0.2,
        "cluster_selection_method": "eom", #eom
        "gen_min_span_tree": True })

#leaf minor
use_classifier_minor_leaves = (hdbscan.HDBSCAN, {"min_cluster_size": 10,
        "min_samples": 100,  #500
        "alpha": 1.0,   #1.0
        "cluster_selection_epsilon": 0.2,
        "cluster_selection_method": "leaf", #eom
        "gen_min_span_tree": True })

#leaf primary
use_classifier_major_leaves = (hdbscan.HDBSCAN, {"min_cluster_size": 100,
        "min_samples": 250,  #500
        "alpha": 1.0,   #1.0
        "cluster_selection_epsilon": 0.35,
        "cluster_selection_method": "leaf", #eom
        "gen_min_span_tree": True })

sclassifier, scategories = localclassify(sample, use_classifier_major_leaves)

print(np.max(scategories))



In [ ]:
vis.seaborn_embedplot(sample, scategories)

In [ ]:
row, col = 100, 200     #y, x
print(kde.Z[row,col])
print(kde.X[row,col])
print(kde.Y[row,col])

In [ ]:
kde.X.shape

In [ ]:
from sklearn.neighbors import KernelDensity
min_separation=config.get('reducer', 'min_separation')
kde_separation_bandwidth_mult=config.get('reducer', 'kde_separation_bandwidth_mult')

for i in range(np.max(scategories)+1):
    indices = np.where(scategories==i)
    selected = embedding[indices]

_kde =  KernelDensity(kernel='gaussian',bandwidth=min_separation*kde_separation_bandwidth_mult)
_kde.fit(selected)





In [ ]:
kde.X[0].shape[0]

In [ ]:
_xy, _X, _Y = clustering.get_linspace(embedding, kde.X[0].shape[0]) 

_Z = _kde.score_samples(_xy)